## **Rough work and everything**

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob, os, re
import matplotlib.pyplot as plt
from datetime import  datetime
from ipywidgets import widgets, interactive

In [2]:
dt = datetime.today().strftime(format='%d-%m-%Y')

### **Establish parent directory**

In [3]:
#uniqueness in directory and file names is assumed for all analyses
sars_dir = "bioinformatics/github/covid/data/SarsGenomics/SARS-CoV-2"
home_dir = os.getenv('HOME')
parent_dir = glob.glob(f'{home_dir}/{sars_dir}', recursive=True)[0]

### **Metadata**

In [4]:
df_meta = pd.read_excel(glob.glob(f'{parent_dir}/**/COVID19-resultsCts-merged-cln.xlsx', recursive=True)[0])
# glob(f'{parent_dir}/**/pc-nc_QC-QA_03-11-2021.xlsx', recursive=True)[0])[['CASE_ID', 'S_NUM']]#, 'G_COV', 'LIN', 'DT_SAM_RECEP', 'DT_SAM_COLL']]

## RF Report August

In [5]:
df_seq_summ = pd.read_excel(glob.glob(f'{parent_dir}/Outputs/seq-summary-metadata_seq52_31-08-2022.xlsx')[0])
df_prev = pd.read_excel(glob.glob(f'{parent_dir}/RFellerReports/rf-20220831_ILRI_gisaid_submission_12_metadata.xls')[0], 
                              sheet_name='Submissions', usecols=['covv_provider_sample_id', 'covv_subm_sample_id'])#covv_subm_sample_id

In [6]:
df_seq_summ. head(2)

,case_id,sequence_name,genome_coverage,lineage,clade,run_num,tech,seq_dt,lib_prep,primer_set,analysis_pipeline,sno.
0,MUS/30826/2022,COVM04097,99.8,BA.2,21L (Omicron),Run52,NextSeq,23/08/2022,COVIDSeq,ARTIC_V3,nf-viralrecon-v2.5,NaN
1,MUS/28727/2022,COVM04048,99.8,BA.2,21L (Omicron),Run52,NextSeq,23/08/2022,COVIDSeq,ARTIC_V3,nf-viralrecon-v2.5,NaN


In [7]:
df_seq_summ.run_num.unique()

array(['Run52'], dtype=object)

In [8]:
mask1 = df_seq_summ.run_num.isin(['Run52'])#, 'Run41'


In [9]:
latest_seq = df_seq_summ[mask1 == True]
print(f'Total sequenced: {latest_seq.shape[0]}')

Total sequenced: 192


In [10]:
mask2, msk1, msk2, msk3 = (latest_seq.case_id.str.contains('SWZ/'), latest_seq.case_id.str.contains('MUS/'),
latest_seq.case_id.str.contains('MUS/'), latest_seq.case_id.str.contains('AGO/'))
# latest_eth = latest_seq[mask2 == True]
# latest_eth.shape[0]#latest_seq.shape[0] - 

In [11]:
print(f'Total submitted: {df_prev.shape[0] - 1}')

Total submitted: 188


In [12]:
mask3 = df_meta.S_NUM.isin(df_prev.covv_subm_sample_id)#covv_subm_sample_id
df_prev.head()

,covv_provider_sample_id,covv_subm_sample_id
0,Sample ID given by originating laboratory,Sample ID given by the submitting laboratory
1,COVM03968,COVM03968
2,COVM03969,COVM03969
3,COVM03970,COVM03970
4,MUS/23958/2022,COVM03972


In [13]:
sub_meta = df_meta[mask3 == True]
sub_meta.head(2)

,abbr,S_NUM,CASE_ID,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,...,SAMP_TYPE,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF,VACC_STATUS,VACC_DOS,RTPCR_POS,AVG_Ct,DT_RUN
30131,30129,COVM03968,NYR/5313/2022,42,M,Farmer,Kenya,Nyeri,Tetu,No,...,NP Swab,2022-07-21 00:00:00,2022-07-21 00:00:00,NaN,2022-08-18 00:00:00,No,NaN,A01,NaN,2022-08-18 14:54:03
30132,30130,COVM03969,NYA/3254/2022,70,F,Farmer,Kenya,Nyandarua,Oljororok,No,...,NP Swab,2022-07-13 00:00:00,2022-07-29 00:00:00,NaN,2022-08-18 00:00:00,Yes,C,A02,NaN,2022-08-18 14:54:03


In [14]:
sub_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 30131 to 30327
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   abbr          188 non-null    object
 1   S_NUM         188 non-null    object
 2   CASE_ID       188 non-null    object
 3   AGE_YRS       187 non-null    object
 4   GEND          188 non-null    object
 5   OCCU          3 non-null      object
 6   NAT           188 non-null    object
 7   COUNT_RES     3 non-null      object
 8   S_COUNT_RES   188 non-null    object
 9   TRAV_HIST     188 non-null    object
 10  TRAV_FROM     0 non-null      object
 11  QUAR_PLACE    1 non-null      object
 12  SYMPS         2 non-null      object
 13  SAMP_TYPE     188 non-null    object
 14  DT_SAM_COLL   188 non-null    object
 15  DT_SAM_RECEP  188 non-null    object
 16  RESULT        0 non-null      object
 17  DT_CONF       188 non-null    object
 18  VACC_STATUS   188 non-null    object
 19  VA

In [15]:
sub_meta[sub_meta.DT_SAM_COLL.isna() == True].shape[0]

0

In [16]:
sub_fnl = sub_meta[sub_meta.DT_SAM_COLL.isna() == False]


In [17]:
sub_fnl.head(2)#.DT_SAM_COLL.astype('datetime64[ns]')

,abbr,S_NUM,CASE_ID,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,...,SAMP_TYPE,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF,VACC_STATUS,VACC_DOS,RTPCR_POS,AVG_Ct,DT_RUN
30131,30129,COVM03968,NYR/5313/2022,42,M,Farmer,Kenya,Nyeri,Tetu,No,...,NP Swab,2022-07-21 00:00:00,2022-07-21 00:00:00,NaN,2022-08-18 00:00:00,No,NaN,A01,NaN,2022-08-18 14:54:03
30132,30130,COVM03969,NYA/3254/2022,70,F,Farmer,Kenya,Nyandarua,Oljororok,No,...,NP Swab,2022-07-13 00:00:00,2022-07-29 00:00:00,NaN,2022-08-18 00:00:00,Yes,C,A02,NaN,2022-08-18 14:54:03


In [18]:
latest_seq = sub_fnl
dt_delta = sub_fnl[~sub_fnl.CASE_ID.str.contains('MUS/')].DT_SAM_COLL.astype('datetime64[ns]') - pd.Timestamp('2022-08-31')
dt_delta1 = sub_fnl[sub_fnl.CASE_ID.str.contains('MUS/')].DT_SAM_COLL.astype('datetime64[ns]') - pd.Timestamp('2022-08-31')
# dt_delta2 = sub_fnl[msk2].DT_SAM_COLL.astype('datetime64[ns]') - pd.Timestamp('2022-06-02')
# dt_delta3 = sub_fnl[msk3].DT_SAM_COLL.astype('datetime64[ns]') - pd.Timestamp('2022-06-02')
print(f'Local: {dt_delta.mean()}')
print(f'Mauritius: {dt_delta1.mean()}')
# print(f'Uganda: {dt_delta2.mean()}')
# print(f'Angola: {dt_delta3.mean()}')

Local: -44 days +00:00:00
Mauritius: -122 days +06:36:58.378378378
